# Configurations

A configuration is essentially a set of (coordinate) frames, where each frame can represent a shape, joint, inertia, etc. This tutorial introduces to basics of creating & loading configurations, the joint and frame state, computing features, and handling the view window.

In [1]:
from robotic import ry
import numpy as np
import time
print(ry.compiled())

compile time: Sep 30 2023 22:39:07


## Adding frames to a configuration

The starting point is to create a `Configuration`:

In [2]:
C = ry.Config()
C.view()

0

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

A configuration is essentially a tree (or forrest) of frames. You usually add models from files, but let's do it manually here.

In [3]:
C.clear()
f = C.addFrame(name='first')
f.setShape(type=ry.ST.marker, size=[.4])
f.setPosition([0.,0.,.5])
f.setQuaternion([1., .3, .0, .0]) #is normalized internally
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: first pos: [0.  0.  0.5] quat: [0.95782629 0.28734789 0.         0.        ]


0

Let's add a second frame, but with first as parent and with a hinge joint!

In [4]:
f = C.addFrame(name='second', parent='first')
f.setJoint(ry.JT.hingeX)
f.setShape(type=ry.ST.marker, size=[.4])
f.setColor([1,0,0])
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: second pos: [0.  0.  0.5] quat: [-0.95782629 -0.28734789 -0.         -0.        ]


0

Since we now have a configuration with a joint, we can articulate it:

In [5]:
q = C.getJointState()
q[0] = q[0] + .1
C.setJointState(q)
print('joint state:', q)
C.view()

joint state: [0.1]


0

Other examples to add:

In [6]:
C.addFrame('ball', 'second') .setShape(ry.ST.sphere, [.1]) .setColor([1.,.5,.0]) .setRelativePosition([-.3,.0,.2])
C.addFrame('box', 'second') .setShape(ry.ST.ssBox, [.3,.2,.1,.02]) .setColor([.5,1.,.0]) .setRelativePosition([.0,.0,.2])
C.addFrame('capsule', 'second') .setShape(ry.ST.capsule, [.3, .05]) .setColor([.0,1.,.5]) .setRelativePosition([.3,.0,.2])
for t in range(100):
    C.setJointState([np.cos(.1*t)])
    C.view()
    time.sleep(.1)

## Loading existing configurations

In [7]:
C.clear()
C.addFile(ry.raiPath('panda/panda.g'))
C.view()

0

Let's add a second panda, but prefix all frame names, and move it to the side

In [8]:
C.addFile(ry.raiPath('panda/panda.g'), 'r_')
base_r = C.getFrame('r_panda_base')
base_r.setPosition([.0, .5, .0])
C.view()

0

We can get the joint state of the full configuration:

In [9]:
print(C.getJointState())
print('joints:', C.getJointNames())
print('frames:', C.getFrameNames())

[ 0.   -1.    0.   -2.    0.    2.    0.    0.05  0.   -1.    0.   -2.
  0.    2.    0.    0.05]
joints: ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_finger_joint1panda_finger_joint2', 'r_panda_joint1', 'r_panda_joint2', 'r_panda_joint3', 'r_panda_joint4', 'r_panda_joint5', 'r_panda_joint6', 'r_panda_joint7', 'r_panda_finger_joint1r_panda_finger_joint2']
frames: ['panda_base', 'panda_link0', 'panda_link0_0', 'panda_joint1_origin', 'panda_joint1', 'panda_link1', 'panda_link1_0', 'panda_joint2_origin', 'panda_joint2', 'panda_link2', 'panda_link2_0', 'panda_joint3_origin', 'panda_joint3', 'panda_link3', 'panda_link3_0', 'panda_joint4_origin', 'panda_joint4', 'panda_link4', 'panda_link4_0', 'panda_joint5_origin', 'panda_joint5', 'panda_link5', 'panda_link5_0', 'panda_joint6_origin', 'panda_joint6', 'panda_link6', 'panda_link6_0', 'panda_joint7_origin', 'panda_joint7', 'panda_link7', 'panda_link7_0', 'panda_joint8_or

Let's animate:

In [10]:
q0 = C.getJointState()
for t in range(20):
    q = q0 + .1*np.random.randn(q0.shape[0])
    C.setJointState(q)
    C.view()
    time.sleep(.2)

## Features: computing geometric properties
For every frame we can query its pose:

In [11]:
f = C.getFrame('r_gripper')
print('gripper pos:', f.getPosition())
print('gripper quat:', f.getQuaternion())
print('gripper rot:', f.getRotationMatrix())

gripper pos: [0.32141215 0.44345913 0.72576189]
gripper quat: [-0.8669439  -0.10911532  0.39042322  0.28995142]
gripper rot: [[ 0.52699576  0.41754092 -0.74022635]
 [-0.58794555  0.80804404  0.03721381]
 [ 0.61367378  0.41560126  0.6713271 ]]


The above provides basic forward kinematics: After `setJointState` you can query the pose of any configuration frame. However, there is a more general way to query *features*, i.e. properties of the configuration in a differentiable manner. You might not use this often; but it is important to understand as these differentiable features are the foundation of how optimization problems are formulated, which you'll need a lot.

Here are some example features to evaluate:

In [12]:
[y,J] = C.eval(ry.FS.position, ['gripper'])
print('position of gripper:', y, '\nJacobian:', J)

position of gripper: [ 0.32842615 -0.00310221  0.76643017] 
Jacobian: [[ 3.10221333e-03  4.33076454e-01 -1.14787661e-02 -1.67755538e-01
  -3.79268649e-03  8.75715712e-02  8.67361738e-19  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.28426146e-01  1.75069738e-02  5.42979997e-01 -1.54572121e-02
   1.81774030e-01 -3.01246858e-03  2.77555756e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.28032821e-01  1.25841880e-02  5.18337387e-01
   3.99599798e-03  2.20150278e-01  1.73472348e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [13]:
# negative(!) distance between two convex shapes (or origin of marker)
C.eval(ry.FS.negDistance, ['panda_coll7', 'r_panda_coll7'])

(array([-0.31946979]),
 array([[ 2.11266970e-01,  2.46852305e-02,  5.50265616e-01,
         -4.20418790e-02,  2.95705159e-02, -1.11343269e-02,
         -2.43657527e-17, -0.00000000e+00, -1.96930432e-01,
         -4.35913337e-02, -5.21550074e-01,  7.40963736e-02,
         -1.50353913e-02,  5.41474581e-03, -5.27869680e-18,
         -0.00000000e+00]]))

In [14]:
# the x-axis of the given frame in world coordinates
C.eval(ry.FS.vectorX, ['gripper'])

(array([ 0.51342582, -0.68935383,  0.51106284]),
 array([[ 0.68935383,  0.51064577,  0.42483481, -0.52659502,  0.332467  ,
         -0.5259756 , -0.45868217,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.51342582,  0.02064269,  0.72089851, -0.02266834, -0.20468464,
         -0.02193998, -0.72379283,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        , -0.48516266,  0.54559434,  0.49845333, -0.61009578,
          0.4988135 , -0.51549452,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]))

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [15]:
C.setJointState(q0)
C.view()

0

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [16]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  7.07106781e-01
  -7.07106781e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01 

Let's do a very questionable thing: adding .1 to all numbers in the frame matrix!

In [17]:
X = X0 + .1
C.setFrameState(X)
C.view()

0

That totally broke the original design of the robot! Setting global frame states overwrites the relative transformations between frames.

(Also, the rows of X have non-normalized quaternions! These are normalized when setting the frame state.)

Let's reset:

In [18]:
C.setFrameState(X0)
C.view()

0

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q. The frame state is not affected by such a selection of active joints.

In [19]:
C.selectJoints(['panda_joint1', 'panda_joint1'])
print('joint state: ', C.getJointState())
print('joint names: ', C.getJointNames() )

joint state:  [0.]
joint names:  ['panda_joint1']


In [20]:
C.selectJoints([], True)
print('joint state: ', C.getJointState())
print('joint names: ', C.getJointNames() )

joint state:  [ 0.      -1.00001  0.      -2.00001  0.       2.00001  0.       0.05
  0.      -1.00001  0.      -2.00001  0.       2.00001  0.       0.05   ]
joint names:  ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_finger_joint1panda_finger_joint2', 'r_panda_joint1', 'r_panda_joint2', 'r_panda_joint3', 'r_panda_joint4', 'r_panda_joint5', 'r_panda_joint6', 'r_panda_joint7', 'r_panda_finger_joint1r_panda_finger_joint2']


## View interaction and releasing objects

You can close and re-open the view window

In [21]:
C.view_close()

In [22]:
# things are still there
C.view(pause=False, message='this is a message')

0

For user interaction it is often useful to wait for a keypress (making `view` a blocking call):

keypressed = C.view(True, 'press some key!')
print('pressed key:', keypressed, chr(keypressed))

Get a screenshot:

In [23]:
img = C.view_getScreenshot()
print(type(img), img.shape)

<class 'numpy.ndarray'> (400, 400, 3)


And release everything, including closing the view

In [24]:
del C